## PBL(1) Project 를 위한 실습(연습)

1. https://agtechresearch.pythonanywhere.com/ 에 접속하여 회원가입해 주세요. (비밀번호는 단순하게 만드는 것을 권장. 예: 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.

In [1]:
project = "housingprice"  # 수정하지 마세요
username = ""  # 회원가입 시 사용한 이메일아이디 (예시. abc@hello.com)
password = ""  # 비밀번호

리더보드 제출을 위한 기본 설정: 아래 코드를 실행해주세요.


In [ ]:
import os
import urllib.request

if not os.path.exists("competition.py"):
    url = "https://raw.githubusercontent.com/agtechresearch/LectureMLbasic/main/competition/competition.py"
    filename = "competition.py"
    urllib.request.urlretrieve(url, filename)

아래 코드를 실행하여 데이터를 다운로드 받습니다: 3개의 csv 파일이 data 폴더에 다운로드됨

 * dataset.csv: 과거 주택매매 데이터 -> 학습에 사용할 데이터셋
 * problem.csv: 현재 A사가 매매를 고려하고 있는 130건의 주택정보 -> ML 모델에 의하여 예측을 수행하여야 할 데이터셋
 * submission.csv: 리더보드 서버 제출을 위한 파일 형식


In [ ]:
import competition

# 파일 다운로드
competition.download_competition_files(project)

100%|██████████| 141k/141k [00:00<00:00, 339kiB/s] 


----------

### 아래는 랜덤포레스트를 사용하여 집값 예측 모델을 만들고, 코랩환경에서 결과를 리더보드에 제출하는 간단한 샘플 코드입니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# Data 경로 설정
DATA_DIR = "data"

In [ ]:
# 학습에 사용할 과거 주택매매 data set 로드 (dataset.csv)
dataset = pd.read_csv(os.path.join(DATA_DIR, "dataset.csv"))

# problem set 로드 (problem.csv)
problemset = pd.read_csv(os.path.join(DATA_DIR, "problem.csv"))

In [ ]:
# 변수가 24개가 있으므로, 모든 변수를 출력하려면 다음과 같이 설정해야 함
pd.options.display.max_columns = 30

dataset  # 학습에 사용할 과거 주택매매 데이터셋 확인: 1340건의 과거 주택매매 데이터셋

,Id,LotArea,Street,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageType,GarageYrBlt,GarageCars,GarageArea,YrSold,SalePrice
0,1,8450,Pave,Inside,7,5,2003,2003,856,856,854,1710,2,1,3,1,8,0,Attchd,2003.0,2,548,2008,208500
1,2,9600,Pave,FR2,6,8,1976,1976,1262,1262,0,1262,2,0,3,1,6,1,Attchd,1976.0,2,460,2007,181500
2,3,11250,Pave,Inside,7,5,2001,2002,920,920,866,1786,2,1,3,1,6,1,Attchd,2001.0,2,608,2008,223500
3,4,9550,Pave,Corner,7,5,1915,1970,756,961,756,1717,1,0,3,1,7,1,Detchd,1998.0,3,642,2006,140000
4,5,14260,Pave,FR2,8,5,2000,2000,1145,1145,1053,2198,2,1,4,1,9,1,Attchd,2000.0,3,836,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,1336,9042,Pave,Inside,7,9,1941,2006,1152,1188,1152,2340,2,0,4,1,9,2,Attchd,1941.0,1,252,2010,266500
1336,1337,9717,Pave,Inside,5,6,1950,1996,1078,1078,0,1078,1,0,2,1,5,0,Attchd,1950.0,1,240,2010,142125
1337,1338,9937,Pave,Inside,5,6,1965,1965,1256,1256,0,1256,1,1,3,1,6,0,Attchd,1965.0,1,276,2008,147500
1338,1339,9600,Pave,Inside,5,7,1940,1950,728,976,332,1308,1,1,2,1,7,2,Detchd,1940.0,1,256,2009,160000


### **<데이터 구성>**

* LotArea : 평방피트 단위의 부지 크기  
* Street : 부동산으로의 도로(street) 접근 유형    
* LotConfig : 지역 구성  

* OverallQual : 전체적인 재료 및 마감 등급  
* OverallCond : 집의 전체적인 상태 등급  
* YearBuilt : 원래 공사일  
* YearRemodAdd : 리모델링 날짜 (리모델링 혹은 추가가 없었던 경우 공사일과 동일)  

* TotalBsmtSF : 지하 총 평방피트  

* 1stFlrSF : 1층 평방피트  
* 2ndFlrSF : 2층 평방피트  

* GrLivArea : 지상 거실 면적 평방 피트  

* FullBath : 지하층 위의 전체 욕실  
* HalfBath : 지하층 위의 반 욕실  
* BedroomAbvGr : 지하층 위의 침실 수 (지하 침실은 포함하지 않음)  
* KitchenAbvGr : 지하층 위의 주방 수  
  
* TotRmsAbvGrd : 지하층 위의 모든 방 (욕실 제외)  
  
* FirePlace : 벽난로 수  

* GarageType : 차고 위치  
* GarageYrBlt : 차고 건설 연도  
  
* GarageCars : 차량 수용 가능 차고 크기  
* GarageArea : 차고 평방피트  

* YrSold : 연 판매  
* SalePrice : 판매 가격

In [ ]:
# problem set 확인: 130건의 문제 데이터셋 (SalePrice를 예측해야 함)
problemset

,Id,LotArea,Street,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageType,GarageYrBlt,GarageCars,GarageArea,YrSold
0,1341,11200,Pave,Inside,5,5,1965,1965,1040,1040,0,1040,1,0,3,1,5,0,Detchd,1965,1,384,2008
1,1342,7200,Pave,Corner,5,7,1951,2000,900,900,0,900,1,0,3,1,5,0,Detchd,2005,2,576,2010
2,1343,16905,Pave,Inside,5,6,1959,1959,1350,1328,0,1328,1,1,2,1,5,2,Attchd,1959,1,308,2007
3,1344,9180,Pave,CulDSac,5,7,1983,1983,840,884,0,884,1,0,2,1,5,0,Attchd,1983,2,504,2007
4,1345,7200,Pave,Inside,5,7,1920,1996,530,581,530,1111,1,0,3,1,6,0,Detchd,1935,1,288,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,1466,11478,Pave,Inside,8,5,2007,2008,1704,1704,0,1704,2,0,3,1,7,1,Attchd,2008,3,772,2010
126,1467,16321,Pave,CulDSac,5,6,1957,1997,1484,1600,0,1600,1,0,2,1,6,2,Attchd,1957,1,319,2006
127,1468,6324,Pave,Inside,4,6,1927,1950,520,520,0,520,1,0,1,1,4,0,Detchd,1920,1,240,2008
128,1469,8500,Pave,Inside,4,4,1920,1950,649,649,668,1317,1,0,3,1,6,0,Detchd,1920,1,250,2008


## 데이터 전처리 및 모델 학습

In [ ]:
# data 와 problem 데이터들을 합쳐서 하나의 데이터로 만들어줍니다 --> 아래의 데이터 전처리 후 다시 분리할 예정
all_data = pd.concat([dataset, problemset], ignore_index=True)

# select_dtypes 함수를 사용하여 데이터 타입이 object(문자형 데이터)인 열을 제외
# select_dtypes 함수는 열에 포함된 데이터들을 type 기준으로 인덱싱 할 수 있도록 해줌
# select_dtypes(include=None, exclude=None) 형태를 가지며, include에 넣은값을 포함하고 exclude에 넣은 값을 제외한 columns(열)을 DaraFrame 형태로 반환함
all_data = all_data.select_dtypes(exclude="object")

# 결측치를 0으로 대체
all_data = all_data.fillna(0)

# Id 열을 제거
all_data = all_data.drop(["Id"], axis=1)

In [ ]:
# 학습 데이터와 문제 데이터로 다시금 분리
train_data = all_data[: len(dataset)]
problem_data = all_data[len(dataset):]

In [ ]:
# 학습 데이터의 SalePrice 열을 제외한 나머지 열을 X로 지정, SalePrice 열을 Y로 지정
X = train_data.drop("SalePrice", axis=1)
Y = train_data["SalePrice"]

In [ ]:
# 모델 학습을 위해 학습 데이터를 80%의 학습 데이터(train)와 20%의 검증 데이터(test)로 나눔
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# 랜덤 포레스트 모델을 사용하여 학습
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(x_train, y_train)

RandomForestRegressor()

In [ ]:
# train 데이터와 test 데이터에 대한 예측값을 구하고 RMSE 값을 계산
from sklearn.metrics import mean_squared_error

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

print("Train RMSE :", mean_squared_error(y_train, train_pred) ** 0.5)
print("Test RMSE :", mean_squared_error(y_test, test_pred) ** 0.5)

Train RMSE : 11317.335440188946
Test RMSE : 32813.96759394546


RMSE: 평균 제곱근 오차

## Problem set 문제에 대한 집값예측 및 리더보드 결과 제출

- 아래 제출 프로세스가 느리다고 중지 후 다시 코드를 여러차례 재실행하는 경우 패널티가 발생할 수 있습니다. (제출 과정에서 제출 횟수 이슈 발생 가능: 하루 최대 200회 까지 가능)
- 제출에 성공할 경우, "제출에 성공하였습니다"의 메세지와 함께 제출 결과 RMSE 가 화면에 출력됩니다.
- 제출결과는 또한 [대회 페이지(리더보드 서버)](https://agtechresearch.pythonanywhere.com/competitions/housingprice/)의 `리더보드` 와 `제출` 탭에서 확인할 수 있습니다.


In [ ]:
# 전처리 과정 중에 SalePrice 가 0으로 채워져 있기 때문에, problem_data 에서 SalePrice를 다시 제거
problem_data = problem_data.drop("SalePrice", axis=1)
# 문제 데이터(problem data)에 대한 예측값을 구함
problem_pred = model.predict(problem_data)

In [ ]:
# 리더보드 서버 제출을 위한 파일 생성
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["SalePrice"] = problem_pred

# 예측 결과 화면에 출력 후 제출
display(submission)
competition.submit(project, username, password, submission)

,Id,SalePrice
0,1341,135517.50
1,1342,127405.44
2,1343,151802.50
3,1344,134928.64
4,1345,98971.36
...,...,...
125,1466,292775.38
126,1467,164557.00
127,1468,81824.87
128,1469,102428.50


아이디:  abc@abc.co.kr
파일명:  submissions/20240504-043231-submission.csv
[제출에 성공하였습니다]
제출 결과: 33588.44806832891
